In [ ]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import glob

In [ ]:
samples = {
    "bff (200GeV)": "bff/*.root",
    "tt": "one/*.root",
    "dy(50_120)": "two/*.root",
    "dy(400_800)": "three/*.root"
}

In [ ]:
!ls *.root

In [ ]:
samples = {
    "EG": "EG_nanoaod_Skim.root",
}

In [ ]:
def get_numpy_cutflow(_upf):
    return  _upf['denis_cutflow_weighted'].values()

def get_values(file_glob, normalize=False, zero_value=-1, density=0):
    files = glob.glob(file_glob)
    for i, root_file in enumerate(files):
        _upf = uproot.open(root_file)
        if i == 0:
            cutflow = get_numpy_cutflow(_upf)
        else:
            cutflow += get_numpy_cutflow(_upf)
    if zero_value > 0: cutflow[0] = zero_value
    if density: cutflow = cutflow/cutflow[0]
    return cutflow

def make_value_dict(sample_dict):
    return {k: get_values(v, zero_value=-1, density=1) for k,v in sample_dict.items()}

In [ ]:
value_dict = make_value_dict(samples)

In [ ]:
def plot_cutflow(ax, values, normalize=False, offset=0, **kwargs):
    cut_names = [
        'Total',
        'HLT step',
        '1st electron',
        '2nd electron',
        'opposite sign electron',
        'no muon',
        'nlowptlep',
        'combined electron cuts (should be identical to 7)',
        'one b-tagged jet',
        'no second jet (if applicable) (CR13)',
    ]
    x = np.arange(len(cut_names))
    if normalize: values = values/values[0]
    ax.bar(x+offset, values, **kwargs)
    ax.set_xticks(x)
    ax.set_xticklabels(cut_names)

In [ ]:
fig, ax = plt.subplots(figsize=(30,10))
n_plots = len(value_dict)
for i, (k,v) in enumerate(value_dict.items()):
    #add boundary between each bin in cutflow
    boundary=.1
    width = (1.0-boundary)/n_plots
    offset = float(i)/n_plots*(1.0-boundary)
    plot_cutflow(ax, v, offset=offset, width = width, label=k)
    print("{}:{}".format(k, v))
ax.legend()
ax.set_yscale('log')
fig.savefig("cutflow_denis.png", facecolor='white')

In [ ]:
cut_names = [
        'Total',
        'HLT step',
        '1st muon',
        '2nd muon',
        'no electron',
        'opposite sign muons',
        'nlowptlep',
        'combined muon cuts (should be identical to 7)',
        'first b-tagged jet',
        'second jet (if applicable) (SR1)',
    ]

In [ ]:
cut_names